In [ ]:
import findspark
findspark.init('/opt/spark') 
import pandas as pd
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [ ]:
!pip install pyspark

In [ ]:
pip install pyspark-cassandra

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col,count,when,isnan
from pyspark.ml.classification import LinearSVC
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.ml.evaluation import ClusteringEvaluator
import pickle
import pandas as pd
from pyspark.sql.functions import to_timestamp, date_format
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import IndexToString
from pyspark.sql.functions import to_json, struct
from kafka import KafkaProducer
import json
from cassandra.cluster import Cluster
import pyspark
from pyspark.sql import SparkSession
from kafka import KafkaConsumer
import json
from pyspark.ml.clustering import KMeansModel
import time
from kafka import KafkaConsumer
from datetime import datetime, timedelta
import uuid
from cassandra.cluster import Cluster
from collections import defaultdict

In [ ]:
print(pyspark.__version__)

In [ ]:
spark=SparkSession.builder.appName('my_app').getOrCreate()
df=spark.read.csv('uber-raw-data-aug14.csv',header=True,inferSchema=True)
df.show()

In [ ]:
print(df)

In [ ]:
from pyspark.sql.functions import split
df = df.withColumn('date', split(df['Date/Time'], ' ')[0]).withColumn('time', split(df['Date/Time'], ' ')[1]).drop('Date/Time')
df = df.select('date','Lat','Lon','Base')
#df.show()

In [ ]:
print(df)

In [ ]:
df = df.withColumn('date_new', to_timestamp('date', 'MM/dd/yyyy'))
df = df.withColumn('year', year('date_new'))
df = df.withColumn('month', month('date_new'))
df = df.withColumn('day', dayofmonth('date_new'))
df = df.drop('date_new')
df = df.drop("date")
#df.show()

In [ ]:
print(df)

In [ ]:
spark = SparkSession.builder \
    .appName("my") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

df.show()

In [ ]:
indexer = StringIndexer(inputCol="Base", outputCol="Base_index")
indexed = indexer.fit(df).transform(df)

In [ ]:
assembler = VectorAssembler(inputCols=['Lat', 'Lon','year','month','day', 'Base_index'], outputCol='features')
data = assembler.transform(indexed)

In [ ]:
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed=0)

In [ ]:
kmeans = KMeans(k=5, seed=100)
model = kmeans.fit(trainingData)

In [ ]:
model.save("kmeans.model")

In [ ]:
data=testData

In [ ]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], 
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

test_data_json = data.select(to_json(struct([data[x] for x in data.columns])).alias("value")).collect()
for data in test_data_json:
    producer.send('test_topic', value=data['value'])
    print(f"{data['value']}")

In [ ]:
consumer = KafkaConsumer('test_topic', bootstrap_servers=['localhost:9092'],
                         auto_offset_reset='earliest', 
                         enable_auto_commit=True, 
                         value_deserializer=lambda m: json.loads(m.decode('utf-8')))
for message in consumer:
    data = message.value
    json_object = json.loads(data)
    df = spark.createDataFrame([(json_object['Lat'], json_object['Lon'],json_object['year'],json_object['month'],json_object['day'],json_object['Base_index'])], ['Lat', 'Lon','year','month','day','Base_index'])                                              
    print(json_object)

In [ ]:
loaded_model = KMeansModel.load("kmeans.model")

consumer = KafkaConsumer('test_topic', bootstrap_servers=['localhost:9092'],
                         auto_offset_reset='earliest', 
                         enable_auto_commit=True, 
                         value_deserializer=lambda m: json.loads(m.decode('utf-8')))
#output_file = "file2.csv"
# with open(output_file, 'w') as file:
#     file.write("features,predictions\n")  

for message in consumer:
    data = message.value
    json_object = json.loads(data)
    df = spark.createDataFrame([(json_object['Lat'], json_object['Lon'],json_object['year'],json_object['month'],json_object['day'],json_object['Base_index'])], ['Lat', 'Lon', 'year', 'month', 'day','Base_index'])
    assembler = VectorAssembler(inputCols=['Lat', 'Lon', 'year', 'month', 'day','Base_index'], outputCol='features')
    df_with_features = assembler.transform(df).select('features')
    index = df_with_features

    predictions = loaded_model.transform(index)
    predictions_df = predictions.toPandas()
#     predictions_df.to_csv(output_file, mode='a', header=False, index=False)

In [ ]:
converter = IndexToString(inputCol="Base_index", outputCol="BaseString")
converted = converter.transform(predictions)
output = converted.select("Lat", "Lon","BaseString" ,"year","month","day", "prediction")

output.show()
output_count = output.count()
print(output_count)

In [ ]:
output.coalesce(1).write.csv("output.csv", header=True)